In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [202]:
import pandas as pd
import numpy as np
import json
import os
from string import digits

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

from molmass import Formula

from main_functions import *

# Molecule database merge with experimental MS/MS data for METASPACE.

This script merges a molecule databae with experimental MS/MS data.  These data can then be submitted to METASPACE to search for nuetral loss and in-source fragments within datasets.

See: /Users/dis/PycharmProjects/word2vec/main_functions.py

In [3]:
# Load reference database by path:
path = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'
ref_db = load_molecule_database(path)

In [4]:
# Parses GNPS_json to GNPS_df:
GNPS_json = '/Users/dis/PycharmProjects/word2vec/ALL_GNPS.json'
GNPS_df = parse_GNPS_json(GNPS_json)

In [5]:
# Identifies subset of GNPS with experimental specrta in ref db:
# Result: # 3920 spectra for 508 can_no_stereo_smiles
GNPS_hits_df = search_GNPS_targets(ref_db, GNPS_df)

In [6]:
GNPS_hits_df.Adduct.value_counts()

M+H        1982
M-H         923
M+Na        464
M+K         219
M+NH4+      176
M-H2O+H     135
M+NH4        13
M+            6
M+Cl          1
M-H2O-H       1
Name: Adduct, dtype: int64

In [7]:
# Identifies subset of Mona with experimental specrta in ref db, runs in a few hours
# Result: # 7646 spectra for 648 inchi matches
# Mona_hits_df = search_MONA(ref_db)

# Reload existing:
Mona_hits_df = pd.read_pickle('mona_2020_Apr_15.pickle')

# Filters Mona_hits_df for high-res instruments, good adducts, and <20 ppm error.
# Result: # 3437 spectra for 535 inchi matches
Mona_hits_df = parse_MONA_out(Mona_hits_df)

In [8]:
Mona_hits_df.adduct.value_counts()

M+H        2805
M+Na        254
M+K         153
M+NH4+      142
M-H2O+H      56
M+           27
Name: adduct, dtype: int64

In [9]:
Mona_hits_df.source.value_counts()

MassBank                                   1197
Vaniya/Fiehn Natural Products Library       836
unknown                                     471
Fiehn HILIC Library                         379
GNPS                                        296
ReSpect                                     199
RIKEN PlaSMA Authentic Standard Library      59
Name: source, dtype: int64

In [22]:
# Cleans up ref db for only entries with experimental MS/MS spectra
df = preparser_Sirius(ref_db, GNPS_hits_df, Mona_hits_df)

In [ ]:
# Runs main Sirius loop
sirius_output_df = loop_Sirius(df, Mona_hits_df, GNPS_hits_df)

In [24]:
# Reload without rerunning
# sirius_output_df.to_pickle('sirius_output_df.pickle')
sirius_output_df = pd.read_pickle('sirius_output_df.pickle')

In [33]:
# Clean-up and merge output with MS2 spectra
clean_output_df = results_clean_up(df, sirius_output_df)

In [ ]:
# Clean-up database for METASPACE
METASPACE_db = output_METASPACE(clean_output_df, out_name)

In [464]:
# Export entire db
METASPACE_db.iloc[:,0:4].to_csv('whole_body_msms_test_v2.csv', sep='\t')

In [ ]:
# Filter for observed MS1 and export:
f2_ids = list(pd.read_csv('f2.txt').moleculeIds)
METASPACE_db = METASPACE_db[METASPACE_db.old_id.isin(f2_ids)]
METASPACE_db.iloc[:,0:4].to_csv('whole_body_msms_test_v3.csv', sep='\t')

In [491]:
# Before filtering 10019 fragments for 776 parents
# After fitering 1228 fragments for 111 parents 

# Friday:
1. Clean-up code 
2. Run Vitally/Lachlan
3. API access ion images and put in folders as pairs